In [12]:
import pandas as pd
import missingno as msno
import seaborn as sns
import os
import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
import emoji
import time
from time import sleep

In [13]:
CARPETA=('./Datos/Originales/')
FICHERO='booking_data_for_cancellation_proyecto.csv'

In [14]:
datos=pd.read_csv(os.path.join(CARPETA,FICHERO),sep=',')

In [26]:
print('5- Missings y outliers')
def quita_na_1(x):
    remplazar='No determinado'
    datos[x]=datos[x].fillna(remplazar)
    return print(f'Se han transformado los valores ausentes de la variable {x} por {remplazar} ')
quita_na_1('cancelled_at')
quita_na_1('travel_agency_name')
quita_na_1('nationality')
def quita_na_2(x):
    datos[x]= datos.groupby([datos['child_count'],datos['adult_count'],datos['nationality']])['age'].transform(lambda x: x.fillna(x.mean()).round())
    return print(f' Se han cambiado los missing values de la variable {x} dependiendo de la nacionalidad y la cantidad de hijos con los que habia reservado')
quita_na_1('age')

5- Missings y outliers
Se han transformado los valores ausentes de la variable cancelled_at por No determinado 
Se han transformado los valores ausentes de la variable travel_agency_name por No determinado 
Se han transformado los valores ausentes de la variable nationality por No determinado 
Se han transformado los valores ausentes de la variable age por No determinado 


In [25]:
def creacion_variables(df):
    print('Diferencia entre la fecha de check in y la de cancelacion')
    df['cancelled_imp']=df['cancelled_at']
    df.loc[df['cancelled_at']=='No determinado', 'cancelled_imp'] = '2000-01-01T00:00:00Z'
    df[df['cancelled_at']!='No determinado']['cancelled_imp']
    df['cancelled_imp'] = pd.to_datetime(df['cancelled_imp'] )
    df['dif_can_in'] = df['checkin_time'] - df['cancelled_imp'] 
    df.loc[df['cancelled_at']=='No determinado', 'dif_can_in'] = 0
    df[df['cancelled_at']!='No determinado']['dif_can_in']

    print('Diferencia entre la fecha de cancelacion y la de reserva')
    df['dif_can_res'] = df['cancelled_imp'] - df['booking_time']
    df.loc[df['cancelled_at']=='No determinado', 'dif_can_res'] = 0
    df[df['cancelled_at']!='No determinado']['dif_can_res']

    print('Diferencia entre la fecha de reserva y la de entrada')
    df['dif_resserva_entrada'] = df['checkin_time'] - df['booking_time']
    print('Analizar la diferenica minima por si puede haber un error')
    df['dif_resserva_entrada'].min() 

    print('Duracion de la reserva:Diferencia entre check in y check out')
    df['duracion_reserva'] = df['checkout_time'] - df['checkin_time']
      

    bins = [16, 18, 25, 35, 50, 65, 80, 90]
    names = ["16-18", "18-24", "25-35", "35-50", "50-65", "65-80", "+80"]
    df['rangos_age'] = pd.cut(df['age'], bins, labels = names)
    df['rangos_age'].value_counts()
    print(f'Rangos de edad {names}')


    print('Total de Personas')
    df['total_personas'] = df['adult_count'] + df['child_count']
    print('Observamos un analiss estadistico para conocer y ver si puede haber algun error')
    df['total_personas'].describe()

    print('Rango de numero de adultos')
    bins = [0, 1, 2, 8]
    names = ["persona sola", "pareja", "grupo de adultos"]
    df['rangos_adultos'] = pd.cut(df['adult_count'], bins, labels = names)
    df['rangos_adultos'].value_counts()

    bins = [-1,0, 1, 2, 5]
    names = ["sin niños", "niño unico", "dos niños", "familia numerosa"]
    df['rangos_niños'] = pd.cut(df['child_count'], bins, labels = names)
    df['rangos_niños'].value_counts()







TypeError: cannot unpack non-iterable int object